In [1]:
import os
import re
import csv
from PIL import Image

In [6]:
# File and Directory Paths
source_dir = "source_files/"
processed_metadata_file = source_dir + "papers_processed.csv"

table_dir = "extracted_tables/"
table_metadata_file = table_dir + "tables.csv"
code_table_dir = table_dir + "table_code/"
image_table_dir = table_dir + "table_images/"

# CSV Size Limit
csv.field_size_limit(300072)

131072

In [3]:
# Table image filtering by max height
def filter_table_image(max_height):
    filtered_tables = []
    for table_image in os.listdir(image_table_dir):
        try:
            image_path = image_table_dir + table_image
            img = Image.open(image_path)
            width, height = img.size
            img.close()
            
            if height > max_height:
                filtered_tables.append(table_image.replace(".png", ""))
                
        except Exception as e:
            print(f"Error for file {table_image}: {e}")
    
    print(f"{len(filtered_tables)} tables have been filtered.")
    return filtered_tables

In [4]:
# Table code filtering by min rows and min columns
def filter_table_code(min_rows, min_cols):
    filtered_tables = []
    for table_code in os.listdir(code_table_dir):
        try:
            rows, cols = count_table_dimension(code_table_dir + table_code)       
            if rows < min_rows or cols < min_cols:
                filtered_tables.append(table_code.replace(".tex", ""))
        except Exception as e:
            print(f"Error for file {table_code}: {e}")
            filtered_tables.append(table_code.replace(".tex", ""))
    print(f"{len(filtered_tables)} tables have been filtered.")
    return filtered_tables 

# Determines the number of rows and columns of a latex table based on its code
def count_table_dimension(code_path):
    f = open(code_path, "r", encoding="utf8")
    latex_code = f.read()
    f.close()
    
    table_content = re.search(r'\\begin\{tabular\}.*?\\end\{tabular\}', latex_code, re.DOTALL)
    if table_content is None:
        raise ValueError("No tabular environment found in the code.")
        
    table_content = table_content.group(0).split("\n",1)[1]
    
    # Remove latex formatting
    table_content = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', table_content)  # Remove commands like \textbf{}
    table_content = re.sub(r'%.*', '', table_content)  # Remove comments
    table_content = re.sub(r"\\(top|mid|bottom)rule", "", table_content)
    table_content = re.sub(r"\\hline", "", table_content)  
    table_content = re.sub(r'\n', ' ', table_content)  # Replace newlines with spaces for simpler processing
    
    # Count rows and columns
    rows = re.split(r'\\\\', table_content)
    rows = [row.strip() for row in rows if row.strip()]
    number_rows = len(rows)
    
    number_columns = 0
    if rows and len(rows) > 1:
        number_columns = rows[1].count('&') + 1
    else:
        print(f"No rows found for {code_path}!")
    
    return number_rows, number_columns

In [5]:
# Removes a subset of tables from the dataset (including table_image, table_code)
def remove_from_dataset(table_subset):
    table_code_files = os.listdir(code_table_dir)
    table_image_files = os.listdir(image_table_dir)
    
    for table in table_subset:
        table_png = table + ".png"
        table_code = table + ".tex"
        
        # Delete image and code files
        if table_png in table_image_files:
            os.remove(image_table_dir + table_png)
        if table_code in table_code_files:
            os.remove(code_table_dir + table_code)
    
    # Remove filtered tables from metadata file
    with open(table_metadata_file, "r", newline='', encoding='utf-8') as input_file:
        with open(table_dir + "tmp.csv", "w", newline='', encoding='utf-8') as output_file:
            csv_reader = csv.reader(input_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csv_writer = csv.writer(output_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
            # Only writes rows of not deleted tables
            for row in csv_reader:
                if row[0] not in table_subset:
                    csv_writer.writerow(row)
                    
    # Replace old csv file with new csv file
    os.replace(table_dir + "tmp.csv", table_metadata_file)

In [7]:
# Max height a table image should have to be not
MAX_HEIGHT = 1700

# Filtering table images
filtered_tables = filter_table_image(MAX_HEIGHT)

# Removing them from the dataset
remove_from_dataset(filtered_tables)

448 tables have been filtered.


In [4]:
# Min rows and colums a latex table should have
MIN_ROWS = 4
MIN_COLUMNS = 4

# Filtering table code
filtered_tables = filter_table_code(MIN_ROWS, MIN_COLUMNS)

# Removing them from the dataset
remove_from_dataset(filtered_tables)

Error for file 2201.00445_TAB_2.tex: No tabular environment found in the code.
Error for file 2201.00445_TAB_3.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2201.01014_TAB_2.tex!
Error for file 2201.03224_TAB_1.tex: No tabular environment found in the code.
Error for file 2201.03224_TAB_2.tex: No tabular environment found in the code.
Error for file 2201.03224_TAB_3.tex: No tabular environment found in the code.
Error for file 2201.03224_TAB_4.tex: No tabular environment found in the code.
Error for file 2201.03224_TAB_5.tex: No tabular environment found in the code.
Error for file 2201.04300_TAB_1.tex: list index out of range
Error for file 2201.05451_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2201.05587_TAB_1.tex!
No rows found for extracted_tables/table_code/2201.05587_TAB_3.tex!
No rows found for extracted_tables/table_code/2201.05756_TAB_2.tex!
Error for file 2201.05761_TAB_3.tex

Error for file 2202.13065_TAB_1.tex: list index out of range
Error for file 2202.13065_TAB_2.tex: list index out of range
Error for file 2202.13065_TAB_3.tex: list index out of range
Error for file 2202.13065_TAB_4.tex: list index out of range
Error for file 2202.13065_TAB_5.tex: list index out of range
Error for file 2202.13065_TAB_6.tex: list index out of range
Error for file 2202.13065_TAB_7.tex: list index out of range
Error for file 2202.13065_TAB_8.tex: list index out of range
No rows found for extracted_tables/table_code/2203.00945_TAB_1.tex!
Error for file 2203.01104_TAB_1.tex: list index out of range
Error for file 2203.01104_TAB_2.tex: list index out of range
Error for file 2203.01104_TAB_3.tex: list index out of range
Error for file 2203.01554_TAB_3.tex: No tabular environment found in the code.
Error for file 2203.01554_TAB_4.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2203.03726_TAB_1.tex!
No rows found for extracted_tables/

No rows found for extracted_tables/table_code/2204.09273_TAB_7.tex!
No rows found for extracted_tables/table_code/2204.09273_TAB_8.tex!
No rows found for extracted_tables/table_code/2204.09273_TAB_9.tex!
Error for file 2204.09345_TAB_1.tex: No tabular environment found in the code.
Error for file 2204.09345_TAB_2.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2204.10192_TAB_2.tex!
Error for file 2204.11811_TAB_1.tex: No tabular environment found in the code.
Error for file 2204.11811_TAB_2.tex: No tabular environment found in the code.
Error for file 2204.11811_TAB_3.tex: No tabular environment found in the code.
Error for file 2204.11811_TAB_4.tex: No tabular environment found in the code.
Error for file 2204.12824_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2204.13346_TAB_2.tex!
No rows found for extracted_tables/table_code/2204.13353_TAB_1.tex!
No rows found for extracted_tables/tabl

No rows found for extracted_tables/table_code/2205.10696_TAB_29.tex!
No rows found for extracted_tables/table_code/2205.10696_TAB_34.tex!
No rows found for extracted_tables/table_code/2205.10696_TAB_37.tex!
No rows found for extracted_tables/table_code/2205.10696_TAB_4.tex!
No rows found for extracted_tables/table_code/2205.10696_TAB_5.tex!
Error for file 2205.10727_TAB_1.tex: list index out of range
Error for file 2205.10727_TAB_2.tex: list index out of range
No rows found for extracted_tables/table_code/2205.11822_TAB_3.tex!
No rows found for extracted_tables/table_code/2205.12035_TAB_2.tex!
No rows found for extracted_tables/table_code/2205.12397_TAB_3.tex!
Error for file 2205.12404_TAB_2.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2205.12404_TAB_6.tex!
No rows found for extracted_tables/table_code/2205.12522_TAB_2.tex!
No rows found for extracted_tables/table_code/2205.12528_TAB_1.tex!
No rows found for extracted_tables/table_code/22

No rows found for extracted_tables/table_code/2206.15055_TAB_1.tex!
No rows found for extracted_tables/table_code/2207.00143_TAB_2.tex!
Error for file 2207.00143_TAB_7.tex: No tabular environment found in the code.
Error for file 2207.00143_TAB_8.tex: No tabular environment found in the code.
Error for file 2207.00143_TAB_9.tex: No tabular environment found in the code.
Error for file 2207.01152_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2207.01152_TAB_2.tex!
No rows found for extracted_tables/table_code/2207.01531_TAB_1.tex!
No rows found for extracted_tables/table_code/2207.01531_TAB_2.tex!
No rows found for extracted_tables/table_code/2207.01531_TAB_3.tex!
No rows found for extracted_tables/table_code/2207.01531_TAB_4.tex!
No rows found for extracted_tables/table_code/2207.03093_TAB_2.tex!
No rows found for extracted_tables/table_code/2207.03116_TAB_2.tex!
Error for file 2207.03116_TAB_3.tex: No tabular environment found in the

...

Error for file 2304.04308_TAB_2.tex: No tabular environment found in the code.
Error for file 2304.04308_TAB_5.tex: No tabular environment found in the code.
Error for file 2304.04308_TAB_6.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2304.04630_TAB_3.tex!
No rows found for extracted_tables/table_code/2304.04853_TAB_1.tex!
Error for file 2304.04864_TAB_3.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2304.05132_TAB_1.tex!
No rows found for extracted_tables/table_code/2304.05132_TAB_2.tex!
Error for file 2304.05679_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2304.06038_TAB_2.tex!
No rows found for extracted_tables/table_code/2304.06038_TAB_5.tex!
Error for file 2304.06090_TAB_4.tex: No tabular environment found in the code.
Error for file 2304.06090_TAB_5.tex: No tabular environment found in the code.
No rows found for extracted_tables/tabl

Error for file 2305.05005_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2305.05042_TAB_2.tex!
No rows found for extracted_tables/table_code/2305.05042_TAB_5.tex!
No rows found for extracted_tables/table_code/2305.05053_TAB_1.tex!
No rows found for extracted_tables/table_code/2305.05053_TAB_2.tex!
No rows found for extracted_tables/table_code/2305.05579_TAB_1.tex!
No rows found for extracted_tables/table_code/2305.05579_TAB_2.tex!
Error for file 2305.06206_TAB_1.tex: No tabular environment found in the code.
Error for file 2305.06206_TAB_2.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2305.06419_TAB_3.tex!
Error for file 2305.06494_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2305.07207_TAB_1.tex!
Error for file 2305.07474_TAB_1.tex: No tabular environment found in the code.
Error for file 2305.07474_TAB_2.tex: No tabular environment f

No rows found for extracted_tables/table_code/2306.04865_TAB_4.tex!
No rows found for extracted_tables/table_code/2306.04865_TAB_5.tex!
No rows found for extracted_tables/table_code/2306.04865_TAB_6.tex!
No rows found for extracted_tables/table_code/2306.04865_TAB_7.tex!
No rows found for extracted_tables/table_code/2306.04865_TAB_8.tex!
No rows found for extracted_tables/table_code/2306.04865_TAB_9.tex!
Error for file 2306.04983_TAB_1.tex: No tabular environment found in the code.
Error for file 2306.04983_TAB_2.tex: No tabular environment found in the code.
Error for file 2306.04983_TAB_3.tex: No tabular environment found in the code.
Error for file 2306.04983_TAB_4.tex: No tabular environment found in the code.
Error for file 2306.05252_TAB_1.tex: No tabular environment found in the code.
Error for file 2306.05318_TAB_1.tex: No tabular environment found in the code.
Error for file 2306.05318_TAB_2.tex: No tabular environment found in the code.
No rows found for extracted_tables/tabl

Error for file 2310.06233_TAB_1.tex: list index out of range
No rows found for extracted_tables/table_code/2310.06271_TAB_3.tex!
No rows found for extracted_tables/table_code/2310.06412_TAB_3.tex!
Error for file 2310.06508_TAB_1.tex: No tabular environment found in the code.
Error for file 2310.06508_TAB_2.tex: No tabular environment found in the code.
Error for file 2310.06517_TAB_1.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2310.06532_TAB_2.tex!
No rows found for extracted_tables/table_code/2310.06582_TAB_2.tex!
No rows found for extracted_tables/table_code/2310.06666_TAB_1.tex!
No rows found for extracted_tables/table_code/2310.06666_TAB_2.tex!
No rows found for extracted_tables/table_code/2310.06666_TAB_3.tex!
Error for file 2310.06666_TAB_4.tex: No tabular environment found in the code.
No rows found for extracted_tables/table_code/2310.06712_TAB_1.tex!
No rows found for extracted_tables/table_code/2310.06803_TAB_1.tex!
No rows fou

NameError: name 'remove_from_dataset' is not defined